# Assignment 1
## Ferry Model | Group 14
### Imports

In [ ]:
import salabim as sim, numpy as np, pandas as pd
import time, sys, random


## Variables & Data

In [ ]:
# Model Variables
SAILING_TIME = sim.Triangular(10,18,13)
NUMBER_OF_CARS = sim.Triangular(70,80,75)
PAYMENT_TIME = sim.Triangular(1,4,2) 
LOADING_TIME = sim.Exponential(1/6) # 10 seconds per car
UNLOADING_TIME = sim.Exponential(5/60) # 5 seconds per time
WAITING_TIME_PREPAID = sim.Exponential(0.5) # minutes
PERCENTAGE_PREPAID = 0.2 

# Model Settings
SIM_TIME = 60*24 # Time in minutes
REPLICATIONS = 10 # Number of experiment replications

# Passenger Data
CAR_NUMBERS = pd.read_csv("TimeTable.csv", sep=";")

## Components
### Class Car
The Car class has three different attributes which influence the selection of a waiting line. A car can either be driven by an employee or a tourist, and - in case a tourist sits in the car - he/she could have prepaid or not. Additionally, the car is either situated on the mainland or on an island.

In [ ]:
class Car(sim.Component):
    def setup(self, cartype, paid, location):
        self.cartype = cartype # either tourist or employee
        self.paid = paid # true for prepaid, false for not prepaid
        self.location = location # either mainland or island
        #self.i = i
        
    
    def animmation_cars(self, id):
        
        if id == "text":
            ao0 = sim.AnimateText(text=self.name(), textcolor='fg', text_anchor='nw')
            return 0, 16, ao0
        else:
            ao0 = sim.AnimateRectangle((-20, 0, 20, 20),
                text=self.name(), fillcolor=id, textcolor='white', arg=self)
            return 45, 0, ao0

    def process(self):  
        # Go to the assigned booth and line depending on the cartype, prepaid and locations
        ## Employee type
        if self.cartype == "employee":
            if self.location == "mainland":
                self.enter(mainland_line1)
            else: 
                self.enter(island_line1)
        ## Tourists
        else: 
            # In case the tourist has prepaid 
            if self.paid:
                if self.location == "mainland":
                    self.enter(mainland_line2)
                else: 
                    self.enter(island_line2)
                    
            # In case the tourist still has to pay
            else: 
                if self.location == "mainland":
                    self.enter(mainland_line3)
                else: 
                    self.enter(island_line3)

## Car Generator
The Car Generator generates entities of the class "Car" according to the time schedule given. 

In [ ]:
class CarGenerator(sim.Component):
    def setup(self, location, cartype):
        self.cartype = cartype # either tourist or employee
        self.location = location # either mainland or island
    
    def process(self):
        #i = 0
        while True:
            #Get current time
            CurrentCarNumbers = CAR_NUMBERS[CAR_NUMBERS["time"]<= (env.now()/60)].tail(1)

            # Wait for the correct amount of time until creating the next car
            ## Get the number of cars
            number_cars = int(CurrentCarNumbers[str(self.cartype) + "_" + str(self.location)])

            ## Calculate the time span the cars arrive in 
            time_span = int(CAR_NUMBERS["time"].iloc[CurrentCarNumbers.index + 1]) - int(CAR_NUMBERS["time"].iloc[CurrentCarNumbers.index])

            ## Check if number of cars is greater than zero, then wait the correct amount of time, otherwise wait for the time interval
            if number_cars > 0:
                # Generate a car
                Car(cartype = self.cartype, paid = (random.random() < PERCENTAGE_PREPAID), location= self.location)

                # Interarrival times are based on an exponential equation
                yield self.hold(sim.Exponential(60*time_span / number_cars))
                
                #i += 1

            else:
                yield self.hold(1)



## Ferry
The ferry is the entity that goes back and forth in between the island and the mainland. It has three main processes, which are
- **load**: 
- **cruise**: 
- **unload** :

In [ ]:
class Ferry(sim.Component):
    def setup(self, capacity, carsonferry, ferryrides, location):
        self.capacity = capacity # indicates how much space there is on the ferry
        self.carsonferry = carsonferry # indicates how many cars there are currently on the ferry 
        self.ferryrides = ferryrides # counts the amount of ferryrides done
        self.location = location # the location of the ferry (either mainland or island)

    def process(self):
        while True: 
            # Load the ferry
            yield from self.load()

            # Wait until both departure time is reached and the ferry is loaded
            yield self.wait(departuretime, ferryloaded, all=True)

            # Cruise the ferry
            yield from self.cruise()

            # Unload the ferry on the other side
            yield from self.unload()


    # Cruising process of the ferry
    def cruise(self):
        # Set the departure time to False
        departuretime.set(False)

        # Add one more ride to the ferry ride attribute
        self.ferryrides += 1

        # Cruise
        yield self.hold(SAILING_TIME.sample())

        # Change the location of the ferry
        if self.location == "mainland":
            self.location = "island"
        else:
            self.location = "mainland"
        

    # Loading process of the ferry
    def load(self):  
        # Determine the capacity
        self.capacity = NUMBER_OF_CARS.sample()

        # As long as there is space left, check the waiting lines and fill up the space
        while self.capacity > self.carsonferry:            
            # Check for departure 
            if departuretime.get():
                break
            
            # Check location
            if self.location == "mainland":
                # Check if there are any cars left in the queues 
                if len(mainland_line1) > 0:
                    self.car = mainland_line1.pop()
                    self.carsonferry += 1
                    yield self.hold(LOADING_TIME.sample())
                elif len(mainland_line2) > 0:
                    self.car = mainland_line2.pop()
                    self.carsonferry += 1
                    yield self.hold(LOADING_TIME.sample())
                elif len(mainland_line3) > 0:
                    self.car = mainland_line3.pop()
                    self.carsonferry += 1
                    yield self.hold(LOADING_TIME.sample())

                # If no cars are left, set the ferryloaded state to true and check again in 5 mins
                else:
                    ferryloaded.set(True)
                    yield self.hold(5)


            # Same goes for the Island
            if self.location == "island":
                if len(island_line1) > 0:
                    self.car = island_line1.pop()
                    self.carsonferry += 1
                    yield self.hold(LOADING_TIME.sample())
                elif len(island_line2) > 0:
                    self.car = island_line2.pop()
                    self.carsonferry += 1
                    yield self.hold(LOADING_TIME.sample())
                elif len(island_line3) > 0:
                    self.car = island_line3.pop()
                    self.carsonferry += 1
                    yield self.hold(LOADING_TIME.sample())
                else:
                    ferryloaded.set(True)
                    yield self.hold(5)

        # When the maximum capacity is reached, set the ferry to loaded
        ferryloaded.set(True)

    # Unloading of the ferry
    def unload(self):
        for i in range(self.carsonferry):
            yield self.hold(UNLOADING_TIME.sample())
        self.carsonferry = 0
        ferryloaded.set(False)

            



## Ferry Operator

In [ ]:
class FerryOperator(sim.Component):
    def process(self):
        while True:
            # Load the ferry       
            CanadianFerry.activate()

            # Wait for 30 mins
            yield self.hold(30)

            # Give the go for departure
            departuretime.set(True)

In [ ]:
# class AnimateMainland_Line1(sim.Animate):
#     def __init__(self):
#         sim.Animate.__init__(
#             self, rectangle0=(-12, -10, 12, 10), x0=300 - 30 * i, y0=100, fillcolor0="red", linewidth0=0
#         )
#     def visible(self, t):
#         return mainland_line1[self.i] is not None

        


# def do_animation():
#     env.animate(True)
#     AnimateMainland_Line1()
#     show_length = sim.Animate(text="", x0=330, y0=100, textcolor0="black", anchor="w")
#     show_length.text = lambda t: "Length= " + str(len(mainland_line1))
    

### Creating and running the environment

In [ ]:
# Create the Environment
env = sim.Environment(time_unit='minutes', trace= True)
env.modelname("Canadian Ferries Simulation")
env.background_color('20%gray')

# Create States
departuretime = sim.State('departuretime', value=False)
ferryloaded = sim.State('ferryloaded', value=False)

# Create Queues
mainland_line1, mainland_line2, mainland_line3 = sim.Queue('mainland_line1'), sim.Queue('mainland_line2'), sim.Queue('mainland_line3')
island_line1, island_line2, island_line3 = sim.Queue('island_line1'), sim.Queue('island_line2'), sim.Queue('island_line3')

# Create a ferry at the beginning of the simulation
CanadianFerry = Ferry(capacity = NUMBER_OF_CARS.sample(), carsonferry = 0, ferryrides = 0, location = "mainland", at=6.5*60)

# Activate the Ferry operators on 6:30
FerryOperator(at=6.5*60)

# Initiate the Car Generators
CarGenerator(cartype="employee", location="island", at = 6*60)
CarGenerator(cartype="employee", location="mainland", at = 6*60)
CarGenerator(cartype="tourist", location="island", at = 6*60)
CarGenerator(cartype="tourist", location="mainland", at = 6*60)

# Animating mainland_lines
sim.AnimateMonitor(mainland_line1.length, x=10, y=450, width=480, height=80, horizontal_scale=5, vertical_scale=5)
sim.AnimateMonitor(mainland_line1.length_of_stay, x=10, y=550, width=480, height=80, horizontal_scale=5, vertical_scale=5)
sim.AnimateText(text=lambda: mainland_line1.length.print_histogram(as_str=True), x=500, y=700,text_anchor='nw', font='narrow', fontsize=10)

sim.AnimateMonitor(mainland_line2.length, x=10, y=250, width=480, height=80, horizontal_scale=5, vertical_scale=5)
sim.AnimateMonitor(mainland_line2.length_of_stay, x=10, y=350, width=480, height=80, horizontal_scale=5, vertical_scale=5)
sim.AnimateText(text=lambda: mainland_line2.length.print_histogram(as_str=True), x=500, y=700,text_anchor='nw', font='narrow', fontsize=10)

sim.AnimateMonitor(mainland_line3.length, x=10, y=50, width=480, height=80, horizontal_scale=5, vertical_scale=5)
sim.AnimateMonitor(mainland_line3.length_of_stay, x=10, y=150, width=480, height=80, horizontal_scale=5, vertical_scale=5)
sim.AnimateText(text=lambda: mainland_line3.length.print_histogram(as_str=True), x=500, y=700,text_anchor='nw', font='narrow', fontsize=10)


# Animate
#do_animation()
env.animate(True)
env.modelname('Demo queue animation')

# Model run
env.run(duration=SIM_TIME)
print()


In [ ]:
island_line1.print_info()
mainland_line3.print_histograms()